In [ ]:
import pandas as pd

main_headers = ['sNo','sPort','rxPkts','rxBytes','rxError','txPkts','txBytes','txError','portSpeed','portStat','linkStat'] 
final_header = [] 
for i in range(1,6):
  for k in range(4):
    for j in main_headers:
      final_header.append(str(i)+str(k)+j)
final_header.extend(['d11','d12','d13','d14','d21','d22','d23','d25','d31','d32','d33','d34','d35','d41','d43','d44','d45','d52','d53','d54','d55','y'])
#print(len(final_header))
#print(final_header)


df = pd.read_csv('alldata.csv',names=final_header)
#print(df.shape)
df.head(3)


import numpy as np

### droping unusefull columns
errors_0_cols = []
delay_0_cols = []
stats_cols = []
for i in df.columns:
  if 'Error' in i:
    errors_0_cols.append(i)
  elif len(i) == 3 and 'd' in i and i[1]==i[2]:
    delay_0_cols.append(i)
  elif 'portStat' in i or 'linkStat' in i:
    stats_cols.append(i)
#print(errors_0_cols)
#print(delay_0_cols)
#print(stats_cols)


sum_errors = df[errors_0_cols].values
sum_delays = df[delay_0_cols].values
#print(sum_errors.shape)
#print(sum_errors.sum())
#print(sum_delays.sum())

delay_0_cols.extend(stats_cols)
errors_0_cols.extend(delay_0_cols)
#print(len(errors_0_cols))


df.drop(errors_0_cols,axis=1,inplace=True)
df.shape


X = df.drop('y',axis=1).values
#print(X.shape)
y = df['y'].values
#print(y.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    stratify=y,
                                                    random_state=0)

X_train1 = X_train
y_train1 = y_train
X_test1 = X_test
y_test1 = y_test

In [ ]:
param_grid_svc = {'linearsvc__C': np.logspace(-3, 2, 6)}
param_grid_lr = {'logisticregression__C':[0.001,.009,0.01,.09,1,5,10,25]}

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs')
linsvc = LinearSVC()

lr_pipe = make_pipeline(StandardScaler(), logreg)
svc_pipe = make_pipeline(StandardScaler(), linsvc)

grid_lr = GridSearchCV(lr_pipe, param_grid = param_grid_lr, cv=5)
grid_svc = GridSearchCV(svc_pipe, param_grid = param_grid_svc, cv=5)

grid_lr.fit(X_train,y_train)
grid_svc.fit(X_train1,y_train1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('linearsvc',
                                        LinearSVC(C=1.0, class_weight=None,
                                                  dual=True, fit_intercept=True,
                                                  intercept_scaling=1,
                                                  loss='squared_hinge',
                                                  max_iter=1000,
                                                  multi_class='ovr',
                                                  penalty='l2',
                                                  random_state=None, tol=0.0001,
                    

In [ ]:
print('lr acc :',grid_lr.score(X_test,y_test))
print('svc acc :',grid_svc.score(X_test1,y_test1))

lr acc : 0.9753231492361927
svc acc : 0.982373678025852


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

y_pred = grid_lr.predict(X_test)
print('=========================== LR =================================')
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
print(precision_recall_fscore_support(y_test, y_pred, average='micro'))
print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))


y_pred1 = grid_svc.predict(X_test1)
print('=========================== SVC ================================')
print(precision_recall_fscore_support(y_test1, y_pred1, average='macro'))
print(precision_recall_fscore_support(y_test1, y_pred1, average='micro'))
print(precision_recall_fscore_support(y_test1, y_pred1, average='weighted'))


'''
'micro':
Calculate metrics globally by counting the total true positives, false negatives and false positives.

'macro':
Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

'weighted':
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label).
This alters ‘macro’ to account for label imbalance;
it can result in an F-score that is not between precision and recall.
'''

=========================== LR =================================
(0.9772148594303738, 0.9759096348015456, 0.9759214814025948, None)
(0.9753231492361927, 0.9753231492361927, 0.9753231492361927, None)
(0.9767389158333545, 0.9753231492361927, 0.9753693717662449, None)
=========================== SVC ================================
(0.9832796081540521, 0.982702350115564, 0.9826593281746807, None)
(0.982373678025852, 0.982373678025852, 0.982373678025852, None)
(0.9831684261769048, 0.982373678025852, 0.9824355123589419, None)


"\n'micro':\nCalculate metrics globally by counting the total true positives, false negatives and false positives.\n\n'macro':\nCalculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.\n\n'weighted':\nCalculate metrics for each label, and find their average weighted by support (the number of true instances for each label).\nThis alters ‘macro’ to account for label imbalance;\nit can result in an F-score that is not between precision and recall.\n"